In [117]:
#импортируем библиотеку pandas
import pandas as pd

In [118]:
#загружаем датасет
pokemon_df = pd.read_csv('../data/pokemon.csv')

In [119]:
#убеждаемся, что все правильно загрузили и заодно смотрим на часть данных в датасете
pokemon_df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [120]:
#смотрим информацию о датасете
pokemon_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


Из данной информации видно, что пропущенные значения есть только в столбце признака "Type 2". В данном случае значения пропущены не из-за того, что о них нет информации, а из-за того, что они попросту отсутствуют у некоторых объектов.

In [121]:
pokemon_df['Type 2'].unique()

array(['Poison', nan, 'Flying', 'Dragon', 'Ground', 'Fairy', 'Grass',
       'Fighting', 'Psychic', 'Steel', 'Ice', 'Rock', 'Dark', 'Water',
       'Electric', 'Fire', 'Ghost', 'Bug', 'Normal'], dtype=object)

In [122]:
pokemon_df['Type 1'].unique()

array(['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric',
       'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice',
       'Dragon', 'Dark', 'Steel', 'Flying'], dtype=object)

Информация, приведенная сверху, говорит нам о том, что оба признака "Type 1" и "Type 2" принимают значения из одного множества значений. Так как это не целевой признак, он является категориальным, мы будем его кодировать с помощью метода One-Hot Encoding. Поэтому сам факт того, что у некоторых покемонов нет второго типа (то есть отсутствуют некоторые значения) не страшен, это все исправится при кодировании.

Теперь необходимо перекодировать категориальные признаки, отчасти для того, чтобы избавиться от пропущенных значений. В данном датасете имеется три категориальных признака: "Name", "Type 1" и "Type 2". Признак "Name" не является информативным, поэтому его можно удалить. Признаки Type 1 и Type 2 закодируем с помощью One-Hot кодирования.

In [123]:
#удаляем столбец Name
pokemon_df = pokemon_df.drop(['Name'], axis=1)

In [124]:
#кодируем признаки с помощью метода One-Hot
type_1_dummies = pd.get_dummies(pokemon_df['Type 1'])
type_2_dummies = pd.get_dummies(pokemon_df['Type 2'])
types_df = pd.DataFrame(index = pokemon_df.index)
types = list(type_2_dummies.columns.values)
for t in types:
    types_df[t] = type_1_dummies[t] + type_2_dummies[t]

pokemon_df = pd.concat([pokemon_df, types_df], sort = False, axis = 1)

#удаляем столбцы Type 1 и Type 2

pokemon_df.drop(["Type 1","Type 2"], axis = 1, inplace = True)

In [125]:
#проверяем датафрейм
pokemon_df.head()

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,...,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,1,318,45,49,49,65,65,45,1,False,...,0,1,0,0,0,1,0,0,0,0
1,2,405,60,62,63,80,80,60,1,False,...,0,1,0,0,0,1,0,0,0,0
2,3,525,80,82,83,100,100,80,1,False,...,0,1,0,0,0,1,0,0,0,0
3,3,625,80,100,123,122,120,80,1,False,...,0,1,0,0,0,1,0,0,0,0
4,4,309,39,52,43,60,50,65,1,False,...,0,0,0,0,0,0,0,0,0,0


In [126]:
pokemon_df.columns

Index(['#', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed',
       'Generation', 'Legendary', 'Bug', 'Dark', 'Dragon', 'Electric', 'Fairy',
       'Fighting', 'Fire', 'Flying', 'Ghost', 'Grass', 'Ground', 'Ice',
       'Normal', 'Poison', 'Psychic', 'Rock', 'Steel', 'Water'],
      dtype='object')

Теперь необходимо закодировать порядковые признаки. В датафрейме имеется два порядковых признака: "#" и "Generation". "#" не является информативным признаком, поэтому его можно удалить. 

In [127]:
#удаляем столбец '#'
pokemon_df = pokemon_df.drop(['#'], axis=1)

Признак "Generation" удобен нам в таком виде, в каком он есть, то есть числовом.

In [128]:
pokemon_df.head()

,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Bug,...,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,318,45,49,49,65,65,45,1,False,0,...,0,1,0,0,0,1,0,0,0,0
1,405,60,62,63,80,80,60,1,False,0,...,0,1,0,0,0,1,0,0,0,0
2,525,80,82,83,100,100,80,1,False,0,...,0,1,0,0,0,1,0,0,0,0
3,625,80,100,123,122,120,80,1,False,0,...,0,1,0,0,0,1,0,0,0,0
4,309,39,52,43,60,50,65,1,False,0,...,0,0,0,0,0,0,0,0,0,0


Обработаем бинарные признаки. Он у нас всего один - 'Legendary'. Переведем False/True в 0/1.

In [129]:
pokemon_df.Legendary.replace({True:1,False:0}, inplace = True)

In [130]:
pokemon_df

,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Bug,...,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,318,45,49,49,65,65,45,1,0,0,...,0,1,0,0,0,1,0,0,0,0
1,405,60,62,63,80,80,60,1,0,0,...,0,1,0,0,0,1,0,0,0,0
2,525,80,82,83,100,100,80,1,0,0,...,0,1,0,0,0,1,0,0,0,0
3,625,80,100,123,122,120,80,1,0,0,...,0,1,0,0,0,1,0,0,0,0
4,309,39,52,43,60,50,65,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,600,50,100,150,100,150,50,6,1,0,...,0,0,0,0,0,0,0,1,0,0
796,700,50,160,110,160,110,110,6,1,0,...,0,0,0,0,0,0,0,1,0,0
797,600,80,110,60,150,130,70,6,1,0,...,1,0,0,0,0,0,1,0,0,0
798,680,80,160,60,170,130,80,6,1,0,...,0,0,0,0,0,0,1,0,0,0


In [131]:
#экспортируем изменения в датафрейме
pokemon_df.to_csv('../data/pokemon_preprocessed.csv', index=False)